# Illustra: Multi-text to Image

Based on [CLIP](https://github.com/openai/CLIP) + FFT from [Lucent](https://github.com/greentfrapp/lucent) // made by [eps696](https://github.com/eps696) [Vadim Epstein]  
thanks to [Ryan Murdock](https://twitter.com/advadnoun), [Jonathan Fly](https://twitter.com/jonathanfly) for ideas

## Features 
* **continuously processes phrase lists** (e.g. illustrating lyrics)
* generates [FFT-encoded](https://github.com/greentfrapp/lucent/blob/master/lucent/optvis/param/spatial.py) image (massive detailed textures, a la deepdream)
* fast convergence
* undemanding for RAM - fullHD/4K and above
* saving/loading FFT snapshots to resume processing
* selectable CLIP model


**Run the cell below after each session restart**

Mark `resume` and upload `.pt` file, if you're resuming from the saved params.

In [ ]:
#@title General setup

import subprocess
CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

try: 
  !pip3 install googletrans==3.1.0a0
  from googletrans import Translator, constants
  translator = Translator()
except: pass
!pip install ftfy==5.8

!apt-get -qq install ffmpeg
from google.colab import drive
drive.mount('/G', force_remount=True)
gdir = !ls /G/
gdir = '/G/%s/' % str(gdir[0])
%cd $gdir
work_dir = 'illustra'
work_dir = gdir + work_dir + '/'
import os
os.makedirs(work_dir, exist_ok=True)
%cd $work_dir

import os
import io
import time
import math
# from math import exp
import random
import imageio
import numpy as np
import PIL
# from skimage import exposure
from base64 import b64encode
import shutil

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable

from IPython.display import HTML, Image, display, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import ipywidgets as ipy
# import glob
from google.colab import output, files

import warnings
warnings.filterwarnings("ignore")

!pip install git+https://github.com/openai/CLIP.git
import clip
!pip install git+https://github.com/Po-Hsun-Su/pytorch-ssim
import pytorch_ssim as ssim

%cd /content
!rm -rf aphantasia
!git clone https://github.com/eps696/aphantasia
%cd aphantasia/
from clip_fft import to_valid_rgb, fft_image, slice_imgs, checkout
from utils import pad_up_to, basename, file_list, img_list, img_read
from progress_bar import ProgressIPy as ProgressBar

clear_output()

resume = False #@param {type:"boolean"}
if resume:
  resumed = files.upload()
  params_pt = list(resumed.values())[0]
  params_pt = torch.load(io.BytesIO(params_pt))

def ema(base, next, step):
    scale_ma = 1. / (step + 1)
    return next * scale_ma + base * (1.- scale_ma)

def save_img(img, fname=None):
  img = np.array(img)[:,:,:]
  img = np.transpose(img, (1,2,0))  
  img = np.clip(img*255, 0, 255).astype(np.uint8)
  if fname is not None:
    imageio.imsave(fname, np.array(img))
    imageio.imsave('result.jpg', np.array(img))

def makevid(seq_dir, size=None):
  out_sequence = seq_dir + '/%05d.jpg'
  out_video = seq_dir + '.mp4'
  !ffmpeg -y -v quiet -i $out_sequence $out_video
  data_url = "data:video/mp4;base64," + b64encode(open(out_video,'rb').read()).decode()
  wh = '' if size is None else 'width=%d height=%d' % (size, size)
  return """<video %s controls><source src="%s" type="video/mp4"></video>""" % (wh, data_url)

!nvidia-smi -L
print('\nDone!')

In [ ]:
#@title Upload text file

translate = False #@param {type:"boolean"}
uploaded = files.upload()

Set the desired video resolution and `duration` (in sec).  
Select CLIP `model` (results do vary!). I prefer ViT for consistency.  

Set `overscan` to produce semi-seamlessly tileable texture (when off, it's more centered).  
Try adding `noise_scale` to explore some compositional changes.  
Decrease `samples` if you face OOM (it's the main RAM eater).  
Increasing `steps` will elaborate details and make tones smoother, but may start throwing texts like graffiti (and will obviously take more time).  
`progressive_grow` may boost macro forms creation (especially with lower `learning_rate`, see more [here](https://github.com/eps696/aphantasia/issues/2)).   
`show_freq` controls preview frequency (doesn't affect the results; one can set it higher to speed up process). 

`--keep X` parameter controls how well the next line/image generation follows the previous. By default X = 0, and every frame is produced independently (i.e. randomly initiated). 
Setting it higher starts each generation closer to the average of previous runs, effectively keeping macro compositions more similar and the transitions smoother. Safe values are < 0.5 (higher numbers may cause the imagery getting stuck). This behaviour depends on the input, so test with your prompts and see what's better in your case.

In [ ]:
#@title Generate

# from google.colab import drive
# drive.mount('/content/GDrive')
# clipsDir = '/content/GDrive/MyDrive/T2I ' + dtNow.strftime("%Y-%m-%d %H%M")

sideX = 1280 #@param {type:"integer"}
sideY = 720 #@param {type:"integer"}
duration =  60#@param {type:"integer"}
#@markdown > Config
model = 'ViT-B/32' #@param ['ViT-B/32', 'RN101', 'RN50x4', 'RN50']
overscan = False #@param {type:"boolean"}
keep = 0. #@param {type:"number"}
noise_scale = 0. #@param {type:"number"}
contrast = 1. #@param {type:"number"}
#@markdown > Training
steps = 200 #@param {type:"integer"}
samples = 200 #@param {type:"integer"}
learning_rate = .05 #@param {type:"number"}
progressive_grow = False #@param {type:"boolean"}
show_freq = 10 #@param {type:"integer"}
fps = 25

model_clip, _ = clip.load(model)
modsize = 288 if model == 'RN50x4' else 224
xmem = {'RN50':0.5, 'RN50x4':0.16, 'RN101':0.33}
if 'RN' in model:
  samples = int(samples * xmem[model])

norm_in = torchvision.transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))

text_file = list(uploaded)[0]
texts = list(uploaded.values())[0].decode().split('\n')
texts = [tt.strip() for tt in texts if len(tt.strip())>0 and tt[0] != '#']
print(' text file:', text_file)
print(' total lines:', len(texts))

workdir = os.path.join(work_dir, basename(text_file))
workdir += '-%s' % model if 'RN' in model.upper() else ''
!rm -rf $workdir
os.makedirs(workdir, exist_ok=True)

outpic = ipy.Output()
outpic
  
# make init
global params_start, params_ema
params_shape = [1, 3, sideY, sideX//2+1, 2]
params_start = torch.randn(*params_shape).cuda() # random init
params_ema = 0.

if resume is True:
  # print(' resuming from', resumed)
  params, _ = fft_image([1, 3, sideY, sideX], resume = params_pt)
  if keep > 0:
    params_ema = params[0].detach()
  torch.save(params_pt, os.path.join(workdir, '000-start.pt'))
else:
  torch.save(params_start, os.path.join(workdir, '000-start.pt'))

torch.save(params_start, 'init.pt') # final init

def process(txt, num):

  global params_start
  params, image_f = fft_image([1, 3, sideY, sideX], resume='init.pt')
  image_f = to_valid_rgb(image_f)
  
  if progressive_grow is True:
    lr1 = learning_rate * 2
    lr0 = lr1 * 0.01
  else:
    lr0 = learning_rate
  optimizer = torch.optim.Adam(params, lr0)
    
  print(' ref text: ', txt)
  if translate:
    translator = Translator()
    txt = translator.translate(txt, dest='en').text
    print(' translated to:', txt)
  tx = clip.tokenize(txt).cuda()
  txt_enc = model_clip.encode_text(tx).detach().clone()
  out_name = '%03d-%s' % (num+1, txt.translate(str.maketrans(dict.fromkeys(list("\n',—|!?/:;\\"), ""))).replace(' ', '_').replace('"', ''))
  tempdir = os.path.join(workdir, out_name)
  !rm -rf $tempdir
  os.makedirs(tempdir, exist_ok=True)

  pbar = ProgressBar(steps) #  // save_freq
  for i in range(steps):
    loss = 0
    
    noise = noise_scale * torch.randn(1, 1, *params[0].shape[2:4], 1).cuda() if noise_scale > 0 else 0.
    img_out = image_f(noise)

    imgs_sliced = slice_imgs([img_out], samples, modsize, norm_in, overscan=overscan, micro=None)
    out_enc = model_clip.encode_image(imgs_sliced[-1])
    loss -= torch.cosine_similarity(txt_enc, out_enc, dim=-1).mean()
    del img_out, imgs_sliced, out_enc; torch.cuda.empty_cache()

    if progressive_grow is True:
      lr_cur = lr0 + (i / steps) * (lr1 - lr0)
      for g in optimizer.param_groups: 
        g['lr'] = lr_cur

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % show_freq == 0:
      with torch.no_grad():
        img = image_f(contrast=contrast).cpu().numpy()[0]
      save_img(img, os.path.join(tempdir, '%05d.jpg' % (i // show_freq)))
      outpic.clear_output()
      with outpic:
        display(Image('result.jpg'))
      del img

    pbar.upd()

  if keep > 0:
    global params_start, params_ema
    params_ema = ema(params_ema, params[0].detach(), num+1)
    torch.save((1-keep) * params_start + keep * params_ema, 'init.pt')

  torch.save(params[0], '%s.pt' % os.path.join(workdir, out_name))
  # shutil.copy(img_list(tempdir)[-1], os.path.join(workdir, '%s-%d.jpg' % (out_name, steps)))
  # os.system('ffmpeg -v warning -y -i %s\%%05d.jpg "%s.mp4"' % (tempdir, os.path.join(workdir, out_name)))
  # HTML(makevid(tempdir))

for i, txt in enumerate(texts):
    process(txt, i)

vsteps = int(duration * fps / len(texts))
tempdir = os.path.join(workdir, '_final')
!rm -rf $tempdir
os.makedirs(tempdir, exist_ok=True)

def read_pt(file):
  return torch.load(file).cuda()

print(' rendering complete piece')
ptfiles = file_list(workdir, 'pt')
pbar = ProgressBar(vsteps * len(ptfiles))
for px in range(len(ptfiles)):
  params1 = read_pt(ptfiles[px])
  params2 = read_pt(ptfiles[(px+1) % len(ptfiles)])

  params, image_f = fft_image([1, 3, sideY, sideX], resume=params1)
  image_f = to_valid_rgb(image_f)

  for i in range(vsteps):
    with torch.no_grad():
      img = image_f((params2 - params1) * math.sin(1.5708 * i/vsteps)**2)[0].permute(1,2,0)
      img = torch.clip(img*255, 0, 255).cpu().numpy().astype(np.uint8)
    imageio.imsave(os.path.join(tempdir, '%05d.jpg' % (px * vsteps + i)), img)
    _ = pbar.upd()

HTML(makevid(tempdir))

